# Подсчет метрик для данных

Некоторые метрики оказались непоказательными и не использовались в исследовании

## Подготовка данных

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [3]:
with open("subtitles_raw/en_raw_0-900.txt", "rt", encoding="utf-8") as f:
    en_corpus_raw = [line.strip() for line in f]

with open("corpora/subtitles/facebook/facebook5_whole.txt", "rt", encoding="utf-8") as f:
    translated_corpus_raw = [line.strip() for line in f]

In [4]:
with open("corpora/subtitles/facebook/facebook5_spacy.bin", "rb") as file:
    en_translated_bytes_data = file.read()

nlp = spacy.blank("en")
doc_bin = DocBin().from_bytes(en_translated_bytes_data)
en_translated_docs = list(doc_bin.get_docs(nlp.vocab))

In [4]:
with open("corpora/subtitles/en_subtitles_spacy_dump.bin", "rb") as file:
    en_bytes_data = file.read()

nlp = spacy.blank("en")
doc_bin = DocBin().from_bytes(en_bytes_data)
en_docs = list(doc_bin.get_docs(nlp.vocab))

In [6]:
en_translated_docs[890000]

Magivers talked about darker areas where the surface looks more solid.

# meteor

In [5]:
import nltk
from nltk.translate import meteor_score

predictions = ['a', 'a', 'a', 'b']
references = [['b', 'a', 'a', 'a']]
meteor_score.meteor_score(references, predictions)

0.9375

In [9]:
meteor_scores = []
for sentence_number in tqdm(range(900000)):
    predictions = [str(token) for token in en_translated_docs[sentence_number]]
    references =  [en_corpus_raw[sentence_number].split()]
    meteor_scores.append(meteor_score.meteor_score(references, predictions))

100%|██████████| 900000/900000 [05:46<00:00, 2596.96it/s]


In [15]:
with open("corpora/subtitles/facebook/facebook5_meteor.txt", "wt", encoding="utf-8") as f:
    for number in meteor_scores:
        f.write(f'{number}\n')

# NIST

In [27]:
from nltk.translate.nist_score import sentence_nist

predictions = ['a', 'a', 'a', 'b', 'a', 'b', 'b', 'b', 'b']
references = [['a', 'a', 'a', 'b', 'a', 'b', 'b', 'b', 'b']]

nist_score = sentence_nist(references, predictions)
nist_score

2.9629065506698953

# BLEU

In [5]:
import evaluate
bertscore = evaluate.load("bertscore")

In [ ]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
results